In [ ]:
!pip install transformers datasets accelerate torch wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!nvidia-smi

Mon Dec 16 01:48:45 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              50W / 400W |  40511MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
import gc

gc.collect()

19221

In [2]:
import os
import gc
from google.colab import drive
import wandb
from datasets import load_from_disk, Dataset
from peft import LoraConfig, TaskType, get_peft_model
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, get_linear_schedule_with_warmup

drive.mount('/content/drive')

MODEL_NAME = "gpt2-medium"
TRAINED_MODELS_DIR = '/content/drive/My Drive/Colab Notebooks/trained_models'
FINE_TUNING_OUTPUT_DIR = '/content/drive/My Drive/Colab Notebooks/fine_tuning_output'
LOG_DIR = '/content/drive/My Drive/Colab Notebooks/logs'
BATCH_SIZE = 64


def get_last_checkpoint(output_dir):
    if not os.path.exists(output_dir):
        return None

    checkpoints = [f for f in os.listdir(output_dir) if f.startswith("checkpoint")]
    checkpoints.sort(key=lambda x: int(x.split('-')[-1]))

    if checkpoints:
        return os.path.join(output_dir, checkpoints[-1])
    return None


def train_fine_tune():
    print("Login no WandB")
    wandb.login(key=os.getenv('WANDB_KEY'))
    print("Carregando o tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    tokenizer.pad_token = tokenizer.eos_token

    tokenized_dataset = load_from_disk(f"{TRAINED_MODELS_DIR}/dataset")
    print("Tokenizer carregado!")

    print("Carregando o modelo pré-treinado...")
    model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
    config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        r=32,
        lora_alpha=32,
        lora_dropout=0.1
    )
    model = get_peft_model(model, config)
    print("Modelo carregado com sucesso!")

    print("Fazendo login no WandB...")
    wandb.login(key=os.getenv('WANDB_KEY'))
    print("Inicializando o projeto no WandB...")
    wandb.init(project="postech03", name="gpt2-medium-fine-tuning")
    print("Projeto inicializado no WandB.")

    print("Configurando os argumentos de treinamento...")
    training_args = TrainingArguments(
        output_dir=FINE_TUNING_OUTPUT_DIR,
        overwrite_output_dir=True,
        eval_strategy="no",
        learning_rate=3e-5,
        per_device_train_batch_size=BATCH_SIZE,
        warmup_steps=100,
        save_steps=100,
        save_total_limit=3,
        logging_dir=LOG_DIR,
        logging_steps=100,
        push_to_hub=False,
        report_to="wandb",
        fp16=True,
        gradient_accumulation_steps=4,
        dataloader_num_workers=2,
        optim="adamw_torch",
    )
    print("Argumentos de treinamento configurados!")

    print("Inicializando o trainer...")
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset,
        processing_class=tokenizer,
    )
    print("Trainer inicializado com sucesso!")

    print("Iniciando o treinamento...")
    last_checkpoint = get_last_checkpoint(FINE_TUNING_OUTPUT_DIR)

    if last_checkpoint:
        print(f"Encontrado checkpoint: {last_checkpoint}")
        trainer.train(resume_from_checkpoint=last_checkpoint)
    else:
        print("Começando do inicio, nenhum checkpoint encontrado")
        trainer.train()

    print("Treinamento concluído!")

    print("Salvando o modelo e o tokenizer...")
    model.save_pretrained(f"{TRAINED_MODELS_DIR}/model/")
    tokenizer.save_pretrained(f"{TRAINED_MODELS_DIR}/tokenizer/")
    print("Modelo e tokenizer salvos com sucesso!")

attempt = 0
while attempt < 3:
  try:
    attempt += 1
    train_fine_tune()
    break
  except Exception as e:
    print(f"Erro treinando {e} na tentativa nº {attempt}, limpando memória e tentando novamente.")
    gc.collect()

print("Treinamento concluído com sucesso!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Login no WandB


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Carregando o tokenizer...


...

train/epoch,▁
train/global_step,▁
total_flos,9.602596959934218e+17
train/epoch,2.99972
train/global_step,15990
train_loss,0
train_runtime,0.0149
train_samples_per_second,275302316.63
train_steps_per_second,1075290.134


Projeto inicializado no WandB.
Configurando os argumentos de treinamento...
Argumentos de treinamento configurados!
Inicializando o trainer...
Trainer inicializado com sucesso!
Iniciando o treinamento...
Encontrado checkpoint: /content/drive/My Drive/Colab Notebooks/fine_tuning_output/checkpoint-15990


/usr/local/lib/python3.10/dist-packages/transformers/trainer.py:3354: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(checkpoint, OPTIMIZER_NAME), map_

Step,Training Loss


Treinamento concluído!
Salvando o modelo e o tokenizer...
Modelo e tokenizer salvos com sucesso!
Treinamento concluído com sucesso!
